# Part 7: Model-based Collaborative Filtering with Alternating Least Squares (ALS) & Evaluation on sample userid 2043
---


## Foreword on Alternating Least Squares (ALS)
---

- Alternating Least Squares (ALS) algorithm is a matrix factorization technique that decomposes user-item interaction matrix (such as user-item ratings matrix for datasets with explicit feedback) into user and item latent factors where their dot product will predict user ratings. It alternates between fixing user or item latent factors to solve for the other via gradient descent at each iteration in the process of minimizing loss. In the course of doing so, it learns these latent user and item factors that predict the user ratings almost just like how a regressor learns coefficients that predict the target, based on input features.

<img src="yelp_data/matfact_explain.png"/>

- ALS can be imported from ```implicit``` library for datasets with implicit feedback (Eg. clickthroughs and page views) or ```pyspark.ml.recommendations``` (which can be used for either implicit or explicit data). Since my data involves user ratings, I will be relying on the pyspark machine learning library.
---

- Please follow the steps below to download and install all the relevant libraries and dependencies BEFORE running this notebook to avoid encountering any errors.

- First up, navigate to your home directory and create a new directory called ```server```:
    ```cd ~```
    ```mkdir server```
  Make sure that the stuff below will be downloaded into this ```server``` folder.
  
- In order to run spark and pyspark on your local machine, kindly ensure that you already have Java installed with the following command in your Terminal or Windows-equivalent in command prompt: ```java -version``` If nothing comes out of this, navigate to this [link](https://java.com/en/download/help/download_options.xml) to download java for mac or windows. You may need to restart your system after installation for java to take effect.

- Next, navigate to this [link](https://www.oracle.com/java/technologies/javase-jdk8-downloads.html) to download the java development kit and then install it.

- Check if scala is installed by executing this command in your Terminal or Windows-equivalent command prompt: ```scala -version```. If nothing comes out, navigate to this [link](https://downloads.lightbend.com/scala/2.11.12/scala-2.11.12.tgz) to download and install scala as well as this [link](https://github.com/sbt/sbt/releases/download/v0.13.17/sbt-0.13.17.tgz) to download and install sbt-0.13.17.tgz.

- Navigate to this [link](https://spark.apache.org/downloads.html) to download Apache Spark. Select the options like the screenshot below and click on "spark-2.4.5-bin-hadoop2.7.tgz" under point 3 to download spark and install it.
<img src="yelp_data/spark_dl.png"/>

- The following should be the directory paths of the software you have downloaded and installed above, where ```HOMEDIRECTORY``` is your home directory's name:
    JDK: ```/Library/Java/JavaVirtualMachines/jdk1.8.0_251.jdk```
    Sbt: ```/Users/HOMEDIRECTORY/server/sbt```
    Scala: ```/Users/HOMEDIRECTORY/server/scala-2.11.12```
    Spark: ```/Users/HOMEDIRECTORY/server/spark-2.4.5-bin-hadoop2.7```

- After all of the above have been installed, set up a ```.bash_profile``` file in your home directory. For Mac users, if you do not already have a ```.bash_profile``` file, navigate to your home directory and create one by executing the following commands:
    ```cd ~```
    ```touch .bash_profile```
    After which, open it with a text editor of your choice and add the following lines of code at the top of the ```.bash_profile``` file, replacing ```HOMEDIRECTORY``` with the name of your home directory:

<img src="yelp_data/spark_bash_profile.png"/>
    
       
- Save and close the ```.bash_profile``` file and execute ```source ~/.bash_profile``` in your Terminal or Windows-equivalent command prompt.

- Completely quit your Terminal and command prompt.

- Now you may proceed to run the rest of the following code.


- ***KINDLY NOTE THAT IF YOU HAVE ENCOUNTERED A CONNECTION REFUSED ERROR OR A JAVA ERROR WHERE IT IS TRYING TO CONNECT TO YOUR IP ADDRESS BUT FAILED WHEN RUNNING ANY PYSPARK-RELATED CELL, KINDLY JUST COPY ALL THE CELLS IN THE NOTEBOOK (HIGHLIGHT THE TOP CELL AND CMD(FOR MAC)/CTRL(FOR WINDOWS) + SHIFT + HIGHLIGHT THE LAST CELL), COPY AND PASTE INTO A FRESH NOTEBOOK AND RUN THEM THERE INSTEAD***

In [ ]:
#this is required to allow pyspark to run in a jupyter notebook
!pip install findspark

In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.ml.feature import StringIndexer, IndexToString
from pyspark.ml import Pipeline as PL
from pyspark.sql.functions import col
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel
from pyspark.sql.functions import max
import pyspark.sql.functions as func
from pyspark.ml import PipelineModel
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, multilabel_confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import joblib

In [3]:
#starting the pyspark session
spark = SparkSession.builder.appName('recommendation_system').getOrCreate()

In [4]:
#reading in curated dataset for model-based collaborative filtering (mbcf)
mbcf_hundten = spark.read.csv('yelp_data/mbcf_hundten.csv', header=True, inferSchema=True)

In [5]:
#looking at how the mbcf looks like for a start
mbcf_hundten.show(3,truncate=True)

+-------------------+-------+-------+
|              shops|ratings|userids|
+-------------------+-------+-------+
|hustle-co-singapore|    5.0|    532|
|hustle-co-singapore|    5.0|   1397|
|hustle-co-singapore|    5.0|   2073|
+-------------------+-------+-------+
only showing top 3 rows



In [6]:
#remember we want to test the ALS rating predictions on userid 2043, just as we did for content-based filtering by the various models earlier on
mbcf_hundten.filter("userids = 2043").show(5)

+--------------------+-------+-------+
|               shops|ratings|userids|
+--------------------+-------+-------+
| hustle-co-singapore|    5.0|   2043|
|benjamin-barker-c...|    4.0|   2043|
|the-coffee-roaste...|    3.0|   2043|
|  koi-cafe-singapore|    5.0|   2043|
|starbucks-singapo...|    4.0|   2043|
+--------------------+-------+-------+
only showing top 5 rows



In [7]:
#this confirmed earlier analyses (in part 1 sub-notebook) that show that userid 2043 rated 980 outlets.
mbcf_hundten.filter("userids = 2043").count()

980

In [8]:
#for ALS recommender algorithm to work, the feature columns must at least be in the numerical format - although some of the feature columns are already in the int format, let's just play safe by converting all except for the ratings' column into the double format.
indexer = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in list(set(mbcf_hundten.columns)-set(['ratings']))]
pipeline = PL(stages=indexer)
transformed = pipeline.fit(mbcf_hundten).transform(mbcf_hundten)
transformed.show(5)

+--------------------+-------+-------+-----------+-------------+
|               shops|ratings|userids|shops_index|userids_index|
+--------------------+-------+-------+-----------+-------------+
| hustle-co-singapore|    5.0|    532|      170.0|         50.0|
| hustle-co-singapore|    5.0|   1397|      170.0|         56.0|
| hustle-co-singapore|    5.0|   2073|      170.0|         90.0|
| hustle-co-singapore|    5.0|   2043|      170.0|          0.0|
|benjamin-barker-c...|    4.0|   2141|      452.0|         92.0|
+--------------------+-------+-------+-----------+-------------+
only showing top 5 rows



In [9]:
#the relevant dtypes are of the correct form now!
transformed.dtypes

[('shops', 'string'),
 ('ratings', 'double'),
 ('userids', 'int'),
 ('shops_index', 'double'),
 ('userids_index', 'double')]

In [10]:
transformed.toPandas().to_csv('pyspark_transformed_mbcf.csv',index=False)

In [11]:
transformed_pandas = pd.read_csv('yelp_data/pyspark_transformed_mbcf.csv')
transformed_pandas.head(3)

,shops,ratings,userids,shops_index,userids_index
0,hustle-co-singapore,5.0,532,170.0,50.0
1,hustle-co-singapore,5.0,1397,170.0,56.0
2,hustle-co-singapore,5.0,2073,170.0,90.0


In [12]:
#the difference in this alternate notebook part 8 is the train_test_split here that was stratified not by ratings but by userids instead since it is logical that the algorithm learns from data (train) on some ratings by a certain group of userids for some outlets and then predict ratings by the same group of userids for other outlets (test)
train, test = train_test_split(transformed_pandas,test_size=0.2,random_state=42,stratify=transformed_pandas['userids'])

In [13]:
train.shape

(2348, 5)

In [14]:
test.shape

(587, 5)

In [16]:
test[(test.userids==75)&(test.shops=='the-coconut-club-singapore')]

,shops,ratings,userids,shops_index,userids_index
1866,the-coconut-club-singapore,5.0,75,148.0,12.0


In [14]:
train.to_csv('train_mbcf_hundten.csv',index=False)
test.to_csv('test_mbcf_hundten.csv',index=False)

In [15]:
training = spark.read.csv('yelp_data/train_mbcf_hundten.csv', header=True, inferSchema=True)
test = spark.read.csv('yelp_data/test_mbcf_hundten.csv', header=True, inferSchema=True)

In [16]:
training.dtypes

[('shops', 'string'),
 ('ratings', 'double'),
 ('userids', 'int'),
 ('userids_index', 'double'),
 ('shops_index', 'double')]

In [17]:
training.count()

2348

In [18]:
test.count()

587

In [19]:
test.dtypes

[('shops', 'string'),
 ('ratings', 'double'),
 ('userids', 'int'),
 ('userids_index', 'double'),
 ('shops_index', 'double')]

In [20]:
#looking at the rating classes (target) in the training dataset to determine baseline accuracy...
training.groupBy("ratings").count().show()

+-------+-----+
|ratings|count|
+-------+-----+
|    1.0|    5|
|    4.0| 1194|
|    3.0|  176|
|    2.0|   15|
|    5.0|  958|
+-------+-----+



<ul>
    
- Baseline accuracy is $\frac{1194\ (majority\ class)}{(5+1194+176+15+958)} = 0.51$ (rounded off to 2 decimal places)

In [21]:
#instantiating the ALS model
als = ALS(maxIter=20,userCol='userids_index',itemCol='shops_index', ratingCol='ratings',coldStartStrategy='drop')

In [22]:
#instantiating the pipeline for tuning of ALS hyperparameters
#pipeline = PL(stages=[als])

In [23]:
#setting up the paramGrid
paramGrid = ParamGridBuilder() \
    .addGrid(als.rank, [200, 300, 400]) \
    .addGrid(als.regParam, [0.01, 0.1, 0.5]) \
    .build()

In [24]:
#setting up the CrossValidator object for hyperparameter tuning
#crossval = CrossValidator(estimator=pipeline,
 #                         estimatorParamMaps=paramGrid,
  #                        evaluator=RegressionEvaluator(predictionCol='prediction',labelCol='ratings',metricName='r2'),
#                          numFolds=5)

In [27]:
#crossval1 = CrossValidator(estimator=ALS(maxIter=20,userCol='userids_index',itemCol='shops_index', ratingCol='ratings',coldStartStrategy='drop'),
 #                         estimatorParamMaps=paramGrid,
  #                        evaluator=RegressionEvaluator(predictionCol='prediction',labelCol='ratings',metricName='r2'),
   #                       numFolds=5)

In [24]:
#setting up the CrossValidator object for hyperparameter tuning
crossval2 = CrossValidator(estimator=als,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator(predictionCol='prediction',labelCol='ratings',metricName='r2'),
                          numFolds=5)

In [25]:
#fitting/training on the train dataset. The following few cells or code have been commented out since this cell took a long time to tune...
#this tuned model yielded best rank of 200 and best regParam of 0.1 as mentioned below...
#cvModel = crossval.fit(training)

In [28]:
#cvModel1 = crossval1.fit(training)

In [2]:
#cvModel2 = crossval2.fit(training)

In [154]:
cvModel2.getEstimatorParamMaps()[np.argmax(cvModel2.avgMetrics)]

{Param(parent='ALS_301793330495', name='rank', doc='rank of the factorization'): 300,
 Param(parent='ALS_301793330495', name='regParam', doc='regularization parameter (>= 0).'): 0.1}

In [168]:
cvModel2.getEstimatorParamMaps()[8]

{Param(parent='ALS_301793330495', name='rank', doc='rank of the factorization'): 300,
 Param(parent='ALS_301793330495', name='regParam', doc='regularization parameter (>= 0).'): 0.1}

In [163]:
#saving model here
cvModel2.save("als_hundten_try1")

In [164]:
#loading the saved model from above cell
cvM1 = CrossValidatorModel.load("yelp_data/als_hundten_try1")

In [166]:
cvModel2.getEstimatorParamMaps()[np.argmax(cvModel2.avgMetrics)]

{Param(parent='ALS_301793330495', name='rank', doc='rank of the factorization'): 300,
 Param(parent='ALS_301793330495', name='regParam', doc='regularization parameter (>= 0).'): 0.1}

<ul>
    
- It gets a little complicated when extracting the best tuned parameters from a pyspark object and thus some of the above code cells may not make sense or be a little messy (as I was experimenting with them) but suffice to note that the best rank and best regParam yielded from the CrossValidator above was 300 and 0.1 respectively...
    
    </ul>

In [1]:
cvM1.getEstimatorParamMaps()[np.argmax(cvM1.avgMetrics)]

In [32]:
#making predictions on the test set
alspredictions = cvModel1.transform(test)

In [33]:
#saving alspredictions to csv for retrieval to test prediction performance later on...
alspredictions.toPandas().to_csv('yelp_data/als_predictions.csv',index=False)

In [19]:
#loading a prefit ALS model containing the best rank = 200 and best regParam = 0.1 derived from the crossval fitted cvModel 4 cells above. Unable to implement the code here due to saving it in the wrong format-and I don't wish to go through the above crossvalidator process again as it took close to a full day just to run it...I could not afford that time as there just wasn't enough time...
#cvModel = ALS.load("yelp_data/als_rec_prefit.model")

In [34]:
#reading in alspredictions to test performance of ALS on test set.
alspreds = spark.read.csv('yelp_data/als_predictions.csv', header=True, inferSchema=True)

In [35]:
#checking out the first few rows of the prediction df
alspreds.show(3)

+--------------------+-------+-------+-----------+-------------+----------+
|               shops|ratings|userids|shops_index|userids_index|prediction|
+--------------------+-------+-------+-----------+-------------+----------+
|the-coconut-club-...|    5.0|     75|      148.0|         12.0|   4.90392|
|    wilder-singapore|    5.0|   2445|      463.0|         64.0|  4.907964|
|the-daily-cut-sin...|    5.0|    225|      496.0|         43.0| 4.9131126|
+--------------------+-------+-------+-----------+-------------+----------+
only showing top 3 rows



In [37]:
#rounding the predictions to a whole number (just like the original or actual ratings) to prepare it for evaluation by MulticlassClassificationEvaluator using f1 score metric
alspreds_df = alspreds.withColumn("prediction_rounded", func.round(alspreds["prediction"],0))

In [38]:
#taking a look at the first few rows of the prediction df with the additional rounded prediction column...
alspreds_df.show(3)

+--------------------+-------+-------+-----------+-------------+----------+------------------+
|               shops|ratings|userids|shops_index|userids_index|prediction|prediction_rounded|
+--------------------+-------+-------+-----------+-------------+----------+------------------+
|the-coconut-club-...|    5.0|     75|      148.0|         12.0|   4.90392|               5.0|
|    wilder-singapore|    5.0|   2445|      463.0|         64.0|  4.907964|               5.0|
|the-daily-cut-sin...|    5.0|    225|      496.0|         43.0| 4.9131126|               5.0|
+--------------------+-------+-------+-----------+-------------+----------+------------------+
only showing top 3 rows



In [56]:
alspreds_df.count()

497

In [39]:
#checking out the data types of the various columns
alspreds_df.dtypes

[('shops', 'string'),
 ('ratings', 'double'),
 ('userids', 'int'),
 ('shops_index', 'double'),
 ('userids_index', 'double'),
 ('prediction', 'double'),
 ('prediction_rounded', 'double')]

In [40]:
#as MulticlassClassificationEvaluator works on double typed values, let's convert rounded predictions to double type.
indexer_1 = StringIndexer(inputCol="prediction_rounded", outputCol='prediction_rounded_dbl')

In [41]:
#implementing the conversion of rounded predictions to double type by fitting and transforming the test set
alspreds_df = indexer_1.fit(alspreds_df).transform(alspreds_df)

In [42]:
#taking a look at the first few rows of the df with the newly added column
alspreds_df.select("shops","ratings","prediction_rounded","prediction_rounded_dbl").show(5,False)

+----------------------------------------------------+-------+------------------+----------------------+
|shops                                               |ratings|prediction_rounded|prediction_rounded_dbl|
+----------------------------------------------------+-------+------------------+----------------------+
|the-coconut-club-singapore                          |5.0    |5.0               |1.0                   |
|wilder-singapore                                    |5.0    |5.0               |1.0                   |
|the-daily-cut-singapore-3                           |5.0    |5.0               |1.0                   |
|kim-kee-coffee-shop-tanjong-rhu-wanton-mee-singapore|4.0    |4.0               |0.0                   |
|andiamo-gelateria-singapore                         |2.0    |2.0               |3.0                   |
+----------------------------------------------------+-------+------------------+----------------------+
only showing top 5 rows



In [43]:
#in order for the double typed rounded predictions to match up with the actual ratings, 
#convert the actual ratings into double type as well for a fairer comparison
indexer_2 = StringIndexer(inputCol="ratings", outputCol='ratings_dbl')

In [44]:
#implementing the conversion
alspreds_df = indexer_2.fit(alspreds_df).transform(alspreds_df)

In [45]:
#checking out the first few rows of the prediction df again
alspreds_df.select("shops","ratings_dbl","prediction_rounded","prediction_rounded_dbl").show(5,False)

+----------------------------------------------------+-----------+------------------+----------------------+
|shops                                               |ratings_dbl|prediction_rounded|prediction_rounded_dbl|
+----------------------------------------------------+-----------+------------------+----------------------+
|the-coconut-club-singapore                          |1.0        |5.0               |1.0                   |
|wilder-singapore                                    |1.0        |5.0               |1.0                   |
|the-daily-cut-singapore-3                           |1.0        |5.0               |1.0                   |
|kim-kee-coffee-shop-tanjong-rhu-wanton-mee-singapore|0.0        |4.0               |0.0                   |
|andiamo-gelateria-singapore                         |3.0        |2.0               |3.0                   |
+----------------------------------------------------+-----------+------------------+----------------------+
only showing top 5 

In [46]:
#instantiating the MulticlassClassificationEvaluator with accuracy score as the metric to compare against baseline accuracy first. 
evaluator_b = MulticlassClassificationEvaluator(predictionCol='prediction_rounded_dbl',
                                                labelCol='ratings_dbl',metricName='accuracy')

In [47]:
#instantiating the MulticlassClassificationEvaluator with f1 score as the metric of choice. 
#F1 score was chosen as both precision and recall are important here-we wouldn't want to miss out 
#on good recommendations and neither do we want to recommend something that shouldn't have been recommended.
evaluator_c = MulticlassClassificationEvaluator(predictionCol='prediction_rounded_dbl',
                                                labelCol='ratings_dbl',metricName='f1')

In [48]:
#accuracy is 1.0!
print("Accuracy is ", evaluator_b.evaluate(alspreds_df))

Accuracy is  1.0


In [49]:
print("F1 score for rating predictions by tuned ALS model-based collaborative filtering: ",evaluator_c.evaluate(alspreds_df))

F1 score for rating predictions by tuned ALS model-based collaborative filtering:  1.0


<ul>
    
- The tuned ALS model performed really well with a high $F_1$ score of 1.0!

In [50]:
#saving alspreds_df with selected columns for reading in again for other evaluation below
alspreds_df.select("shops","ratings","prediction_rounded", "prediction","userids").toPandas().to_csv("yelp_data/mbcf_ALS.csv",index=False)

In [51]:
#reading in mbcf_ALS for other evaluation
mbcf_als_2 = pd.read_csv("yelp_data/mbcf_ALS.csv")
mbcf_als_2.head(3)

,shops,ratings,prediction_rounded,prediction,userids
0,the-coconut-club-singapore,5.0,5.0,4.903920,75
1,wilder-singapore,5.0,5.0,4.907964,2445
2,the-daily-cut-singapore-3,5.0,5.0,4.913113,225


In [54]:
#checking out the value_counts of rounded predictions
mbcf_als_2['prediction_rounded'].value_counts()

4.0    253
5.0    206
3.0     35
2.0      2
1.0      1
Name: prediction_rounded, dtype: int64

In [55]:
#checking out the value_counts of actual ratings
mbcf_als_2['ratings'].value_counts()

4.0    253
5.0    206
3.0     35
2.0      2
1.0      1
Name: ratings, dtype: int64

In [40]:
#saving as csv and then re-reading it to use spark evaluator to recalculate the F1 score...
#mbcf_als.to_csv('yelp_data/mbcf_ALS_mod.csv',index=False)

In [41]:
#reading in the modified "mbcf_ALS" csv for evaluation of F1 using multiclassclassificationevaluator
#df2_mod = spark.read.csv('yelp_data/mbcf_ALS_mod.csv', header=True, inferSchema=True)

In [42]:
#in order for the modified prediction_rounded and actual ratings to be recognized by the multiclassclassification evaluator, 
#convert the both columns into double type again
#indexer_3 = StringIndexer(inputCol="ratings", outputCol='ratings_double_pre')

In [43]:
#implementing the conversion
#df3_mod = indexer_3.fit(df2_mod).transform(df2_mod)

In [44]:
#instantiating the indexer 4 to convert prediction_rounded column to double precision
#indexer_4 = StringIndexer(inputCol="prediction_rounded", outputCol='prediction_rounded_double_pre')

In [45]:
#implementing the conversion
#df4_mod = indexer_4.fit(df3_mod).transform(df3_mod)

In [82]:
#instantiating the MulticlassClassificationEvaluator with accuracy score to compare against baseline accuracy first- see if correcting some of the incorrect rating classes have changed the accuracy score
#evaluator_c_1 = MulticlassClassificationEvaluator(predictionCol='prediction_rounded_double_pre',
                                               # labelCol='ratings_double_pre',metricName='accuracy')

In [46]:
#instantiating the MulticlassClassificationEvaluator with f1 score as the metric of choice. 
#F1 score was chosen as both precision and recall are important here-we wouldn't want to miss out 
#on good recommendations and neither do we want to recommend something that shouldn't have been recommended.
#evaluator_d = MulticlassClassificationEvaluator(predictionCol='prediction_rounded_double_pre',
                                            #    labelCol='ratings_double_pre',metricName='f1')

In [83]:
#accuracy after amendment of rating class encoding is still 0.97!
#print("Accuracy is ", evaluator_c_1.evaluate(df4_mod))

Accuracy is  0.9685916919959473


In [47]:
#F1 score remained unchanged!
#print("F1 score for slightly-modified rating predictions by tuned ALS model-based collaborative filtering: ",evaluator_d.evaluate(df4_mod))

F1 score for slightly-modified rating predictions by tuned ALS model-based collaborative filtering:  0.9820064562401125


In [57]:
#having a rough look at the confusion matrix for model-based collaborative filtering via ALS...
multilabel_confusion_matrix(mbcf_als_2.ratings,mbcf_als_2.prediction_rounded)

array([[[496,   0],
        [  0,   1]],

       [[495,   0],
        [  0,   2]],

       [[462,   0],
        [  0,  35]],

       [[244,   0],
        [  0, 253]],

       [[291,   0],
        [  0, 206]]])

In [58]:
#apart from rating class 1.0, the ALS model seemed to be able to predict the rating classes well!
print(classification_report(mbcf_als_2.ratings,mbcf_als_2.prediction_rounded))

              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00         1
         2.0       1.00      1.00      1.00         2
         3.0       1.00      1.00      1.00        35
         4.0       1.00      1.00      1.00       253
         5.0       1.00      1.00      1.00       206

    accuracy                           1.00       497
   macro avg       1.00      1.00      1.00       497
weighted avg       1.00      1.00      1.00       497



## Defining functions for evaluation of model (confusion matrix, micro-average precision, recall)
---

In [59]:
#defining function for obtaining tn, fp, fn, tp for each rating class for feeding into micro-avg precision and recall functions defined below
def cm_spec(y_true,y_pred,rating,state):
    if state=='tn':
        return multilabel_confusion_matrix(y_true,y_pred)[rating-1][0][0]
    elif state=='fp':
        return multilabel_confusion_matrix(y_true,y_pred)[rating-1][0][1]
    elif state=='fn':
        return multilabel_confusion_matrix(y_true,y_pred)[rating-1][1][0]
    else:
        return multilabel_confusion_matrix(y_true,y_pred)[rating-1][1][1]
    

In [60]:
#defining function for obtaining micro-avg precision
def micro_avg_precision(y_true,y_pred):
    return ((cm_spec(y_true,y_pred,1,'tp')+
                                                 cm_spec(y_true,y_pred,2,'tp')+
                                                 cm_spec(y_true,y_pred,3,'tp')+
                                                 cm_spec(y_true,y_pred,4,'tp')+
                                                 cm_spec(y_true,y_pred,5,'tp'))/(
                                                cm_spec(y_true,y_pred,1,'tp')+
                                                 cm_spec(y_true,y_pred,2,'tp')+
                                                 cm_spec(y_true,y_pred,3,'tp')+
                                                 cm_spec(y_true,y_pred,4,'tp')+
                                                 cm_spec(y_true,y_pred,5,'tp')+
                                                cm_spec(y_true,y_pred,1,'fp')+
                                                 cm_spec(y_true,y_pred,2,'fp')+
                                                 cm_spec(y_true,y_pred,3,'fp')+
                                                 cm_spec(y_true,y_pred,4,'fp')+
                                                 cm_spec(y_true,y_pred,5,'fp')))

In [61]:
#defining function for obtaining micro-avg recall
def micro_avg_recall(y_true,y_pred):
    return ((cm_spec(y_true,y_pred,1,'tp')+
                                                 cm_spec(y_true,y_pred,2,'tp')+
                                                 cm_spec(y_true,y_pred,3,'tp')+
                                                 cm_spec(y_true,y_pred,4,'tp')+
                                                 cm_spec(y_true,y_pred,5,'tp'))/(
                                                cm_spec(y_true,y_pred,1,'tp')+
                                                 cm_spec(y_true,y_pred,2,'tp')+
                                                 cm_spec(y_true,y_pred,3,'tp')+
                                                 cm_spec(y_true,y_pred,4,'tp')+
                                                 cm_spec(y_true,y_pred,5,'tp')+
                                                cm_spec(y_true,y_pred,1,'fn')+
                                                 cm_spec(y_true,y_pred,2,'fn')+
                                                 cm_spec(y_true,y_pred,3,'fn')+
                                                 cm_spec(y_true,y_pred,4,'fn')+
                                                 cm_spec(y_true,y_pred,5,'fn')))

In [62]:
#defining function for obtaining micro_avg_f1
def micro_avg_f1(y_true,y_pred):
    return 2 * ((micro_avg_precision(y_true,y_pred) * micro_avg_recall(y_true,y_pred))/(micro_avg_precision(y_true,y_pred) + micro_avg_recall(y_true,y_pred)))

In [63]:
#function to print out confusion matrix breakdown for each rating class
def confusion_breakdown(y_true,y_pred,rating):
    print("True negatives for rating {}: {}".format(
        rating,multilabel_confusion_matrix(y_true,y_pred)[rating-1][0][0]))
    print("False positives for rating {}: {}".format(
        rating,multilabel_confusion_matrix(y_true,y_pred)[rating-1][0][1]))
    print("False negatives for rating {}: {}".format(
        rating,multilabel_confusion_matrix(y_true,y_pred)[rating-1][1][0]))
    print("True positives for rating {}: {}".format(
        rating,multilabel_confusion_matrix(y_true,y_pred)[rating-1][1][1]))
    return "******************************************"

In [64]:
print(confusion_breakdown(mbcf_als_2.ratings,mbcf_als_2.prediction_rounded,1))
print(confusion_breakdown(mbcf_als_2.ratings,mbcf_als_2.prediction_rounded,2))
print(confusion_breakdown(mbcf_als_2.ratings,mbcf_als_2.prediction_rounded,3))
print(confusion_breakdown(mbcf_als_2.ratings,mbcf_als_2.prediction_rounded,4))
print(confusion_breakdown(mbcf_als_2.ratings,mbcf_als_2.prediction_rounded,5))

True negatives for rating 1: 496
False positives for rating 1: 0
False negatives for rating 1: 0
True positives for rating 1: 1
******************************************
True negatives for rating 2: 495
False positives for rating 2: 0
False negatives for rating 2: 0
True positives for rating 2: 2
******************************************
True negatives for rating 3: 462
False positives for rating 3: 0
False negatives for rating 3: 0
True positives for rating 3: 35
******************************************
True negatives for rating 4: 244
False positives for rating 4: 0
False negatives for rating 4: 0
True positives for rating 4: 253
******************************************
True negatives for rating 5: 291
False positives for rating 5: 0
False negatives for rating 5: 0
True positives for rating 5: 206
******************************************


In [65]:
#all correct!
mbcf_als_2.ratings.value_counts()

4.0    253
5.0    206
3.0     35
2.0      2
1.0      1
Name: ratings, dtype: int64

In [66]:
print("Tuned ALS yielded micro_avg_precision of ", micro_avg_precision(mbcf_als_2.ratings,mbcf_als_2.prediction_rounded))

Tuned ALS yielded micro_avg_precision of  1.0


In [67]:
print("Tuned ALS yielded micro_avg_recall of ", micro_avg_recall(mbcf_als_2.ratings,mbcf_als_2.prediction_rounded))

Tuned ALS yielded micro_avg_recall of  1.0


In [68]:
print("Tuned ALS yielded micro_avg_f1 of ", micro_avg_f1(mbcf_als_2.ratings,mbcf_als_2.prediction_rounded))

Tuned ALS yielded micro_avg_f1 of  1.0


<ul>
    
- Seems like the $F_1$ score computed outside of spark for ALS is also around 1.0! 

In [64]:
#filtering out only userid2043 for comparison with content-based filtering later on
#userid2043_mbcf_test_rev = mbcf_als_final_corrected[mbcf_als_final_corrected['userids']==2043][['shops','ratings','prediction_rounded','prediction']]
#userid2043_mbcf_test_rev.head()

,shops,ratings,prediction_rounded,prediction
0,the-bao-makers-singapore,4.0,4.0,3.933102
2,little-farms-cafe-singapore,4.0,4.0,3.895577
3,lam-yeo-coffee-powder-fty-singapore,5.0,5.0,4.869991
17,stateland-cafe-singapore,5.0,5.0,4.882576
19,coffee-club-singapore-5,5.0,5.0,4.850962


In [65]:
#userid2043_mbcf_test_rev.shape

(185, 4)

In [69]:
#saving this df to csv for easier retrieval later on for comparison and fusion with content-based filtering's rating predictions
mbcf_als_2.to_csv('yelp_data/mbcf_hundten_pred_actual_testeval.csv', index=False)

In [67]:
#old cells, kept for ref only
#df2.show(5,False)

In [68]:
#old cells, kept for ref only..
#having a look at how some of the predictions compare with the actual ratings
#df2.filter("userids = 2043").select("shops","ratings","prediction_rounded","prediction").sort("prediction", ascending=False).show(5, False)

In [69]:
#old cells, kept for ref only...
#filtering out only userid2043 for comparison with content-based filtering later on, converting to pandas df for easier manipulation and merging with content-based rating predictions
#userid2043_mbcf_test = df2.filter("userids = 2043").select("shops","ratings","prediction_rounded","prediction").toPandas()

In [70]:
#old cells, kept for ref only...
#taking a look at the first few rows of the prediction vs actual ratings ("ratings") for ALS tuned with CrossValidator
#userid2043_mbcf_test.head()

In [71]:
#old cells, kept for ref only...
#userid2043_mbcf_test.shape

In [72]:
#saving this df to csv for easier retrieval later on for comparison and fusion with content-based filtering's rating predictions
#userid2043_mbcf_test.to_csv('yelp_data/userid2043_mbcf_pred_actual.csv', index=False)

In [73]:
#old method that would have worked had I saved the cvModel up above correctly...
#extracting the index of the best params
#best_model_params_idx = np.argmax(cvModel.avgMetrics)

In [74]:
#old method that would have worked, see above cell
#extracting the tuned hyperparameters of the best model
#best_rank = list(cvModel.getEstimatorParamMaps()[best_model_params_idx].values())[0]
#best_regParam = list(cvModel.getEstimatorParamMaps()[best_model_params_idx].values())[1]

<ul>
    
- The best rank is 200 while best regParam is 0.1. Fed these into the ALS instantiation below and saved it for future use.

In [78]:
#instantiating ALS with the tuned hyperparameters of the best model
#als_rec = ALS(rank=best_rank,regParam=best_regParam,
#              maxIter=20,userCol='userids_index',
#              itemCol='shops_index', ratingCol='ratings',
#              coldStartStrategy='drop')

In [79]:
#dumping the instantiated model above for later use in another jupyter notebook
#als_rec.save("yelp_data/als_rec_prefitted.model")

## Model Summary and Result Interpretation
---

<ul>
    
- Except for rating 1 which contains a number of false positives, the tuned model-based ALS was able to predict the rating classes well!
- Accuracy of 0.97, Micro-Averaged precision of 0.97, Micro-Averaged recall of 0.97, Micro-Averaged $F_1$ of 0.97

## Source(s)
---

- https://java.com/en/download/help/download_options.xml
- https://www.oracle.com/java/technologies/javase-jdk8-downloads.html
- https://downloads.lightbend.com/scala/2.11.12/scala-2.11.12.tgz
- https://github.com/sbt/sbt/releases/download/v0.13.17/sbt-0.13.17.tgz
- https://spark.apache.org/downloads.html
- https://medium.com/luckspark/installing-spark-2-3-0-on-macos-high-sierra-276a127b8b85